In [1]:
!pip install torch transformers datasets pandas scikit-learn pythainlp

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import load_dataset
import random

# --- ส่วนของการตั้งค่าเพื่อจำลองข้อมูล ---
# กำหนด Aspect และ Keywords ที่เกี่ยวข้อง (ในบริบทของร้านอาหาร)
ASPECT_KEYWORDS = {
    'รสชาติ': ['อร่อย', 'รสชาติ', 'หวาน', 'เค็ม', 'จืด', 'แซ่บ', 'กลมกล่อม'],
    'ราคา': ['ราคา', 'แพง', 'ถูก', 'คุ้ม', 'จ่าย'],
    'บริการ': ['บริการ', 'พนักงาน', 'เจ้าของร้าน', 'ช้า', 'เร็ว', 'ใส่ใจ'],
    'บรรยากาศ': ['บรรยากาศ', 'ร้านสวย', 'น่านั่ง', 'สะอาด', 'วิวดี', 'ร้อน']
}

# กำหนด Sentiment และ Keywords ที่เกี่ยวข้อง
SENTIMENT_KEYWORDS = {
    'positive': ['ดี', 'อร่อย', 'เยี่ยม', 'ชอบ', 'ประทับใจ', 'คุ้ม', 'เร็ว', 'สวย'],
    'negative': ['แย่', 'ห่วย', 'แพง', 'ช้า', 'ไม่ดี', 'สกปรก', 'ร้อน'],
    'neutral': ['เฉยๆ', 'ธรรมดา', 'พอใช้', 'ปกติ']
}

print("กำลังโหลด Wongnai Reviews Dataset...")
# โหลดชุดข้อมูลจาก Hugging Face
dataset = load_dataset("wongnai/wongnai_reviews", split='train')
df = dataset.to_pandas()

print(f"โหลดข้อมูลดิบมาทั้งหมด {len(df)} รีวิว")

# --- สร้างข้อมูลสำหรับฝึกซ้อม (train_data.csv) ---
training_samples = []
# กลับด้าน Dictionary เพื่อให้ค้นหา Aspect จาก Keyword ได้ง่ายขึ้น
keyword_to_aspect = {keyword: aspect for aspect, keywords in ASPECT_KEYWORDS.items() for keyword in keywords}

# กลับด้าน Dictionary เพื่อให้ค้นหา Sentiment จาก Keyword ได้ง่ายขึ้น
keyword_to_sentiment = {keyword: sentiment for sentiment, keywords in SENTIMENT_KEYWORDS.items() for keyword in keywords}

for index, row in df.iterrows():
    review = row['review_body']
    found_aspects_in_review = set()

    # วนลูปหา keyword ของ aspect และ sentiment ในรีวิว
    for aspect_kw, aspect in keyword_to_aspect.items():
        if aspect_kw in review:
            # เมื่อเจอ aspect หนึ่งแล้ว ให้หา sentiment ต่อ
            for sentiment_kw, sentiment in keyword_to_sentiment.items():
                if sentiment_kw in review:
                    # เพื่อไม่ให้มีข้อมูลซ้ำซ้อนมากเกินไป, 1 รีวิวจะถูกใช้สำหรับ 1 aspect เท่านั้น
                    if aspect not in found_aspects_in_review:
                        training_samples.append({
                            'review_text': review,
                            'aspect': aspect,
                            'sentiment': sentiment
                        })
                        found_aspects_in_review.add(aspect)

# สร้าง DataFrame
train_df = pd.DataFrame(training_samples)
# สุ่มข้อมูลเพื่อให้ข้อมูลกระจายตัว
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# --- สร้างข้อมูลสำหรับทดสอบ (test_data.csv) ---
# ใช้ข้อมูลส่วนที่เหลือที่ไม่ได้ถูกเลือกไปเป็นข้อมูล train
remaining_reviews = df[~df['review_body'].isin(train_df['review_text'])]
test_df = remaining_reviews.sample(n=500, random_state=42).reset_index(drop=True)
test_df = test_df[['review_body']].rename(columns={'review_body': 'review_text'})
test_df['review_id'] = test_df.index

# บันทึกไฟล์
train_df.to_csv("train_data.csv", index=False, encoding='utf-8-sig')
test_df.to_csv("test_data.csv", index=False, encoding='utf-8-sig')

print(f"สร้าง train_data.csv สำเร็จ! มีข้อมูล {len(train_df)} ตัวอย่าง")
print(f"สร้าง test_data.csv สำเร็จ! มีข้อมูล {len(test_df)} ตัวอย่าง")
print("\nตัวอย่างข้อมูล Train:")
print(train_df.head())


กำลังโหลด Wongnai Reviews Dataset...
โหลดข้อมูลดิบมาทั้งหมด 40000 รีวิว
สร้าง train_data.csv สำเร็จ! มีข้อมูล 85557 ตัวอย่าง
สร้าง test_data.csv สำเร็จ! มีข้อมูล 500 ตัวอย่าง

ตัวอย่างข้อมูล Train:
                                         review_text    aspect sentiment
0  บ้านน่านเป็นโรงแรมขนาดไม่ใหญ่ มีห้องอาหารอยู่ช...    รสชาติ  positive
1  ลุงสนั่นหมูกระทะเป็นการขายแบบเป็นชุดๆ\nโดยเริ่...  บรรยากาศ  positive
2  เคยมาเที่ยวไหว้พระธาตุไชยาเมื่อปีที่แล้วตอนสงก...    บริการ  positive
3  บรรยากาศร้านสวย น่านั่งชิวๆ เดินทางสะดวก\nราคา...  บรรยากาศ  positive
4  ใครที่ชอบอาหารจีนแนวภัตตาคารละก็ห้ามพลาดร้านนี...      ราคา  positive


### Aspect Extractor (NER)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW
import pandas as pd
from sklearn.model_selection import train_test_split

# --- การตั้งค่าพื้นฐาน ---
MODEL_NAME = 'airesearch/wangchanberta-base-att-spm-uncased'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# โหลดข้อมูลที่สร้างไว้
df = pd.read_csv("train_data.csv")

# --- เตรียมข้อมูลสำหรับ NER ---
# สร้าง Label Map สำหรับ BIO Tagging
# B-ASPECT: Begin of Aspect, I-ASPECT: Inside of Aspect, O: Outside
unique_aspects = df['aspect'].unique().tolist()
labels_to_ids = {'O': 0}
for aspect in unique_aspects:
    labels_to_ids[f'B-{aspect}'] = len(labels_to_ids)
    labels_to_ids[f'I-{aspect}'] = len(labels_to_ids)
ids_to_labels = {v: k for k, v in labels_to_ids.items()}
NUM_NER_LABELS = len(labels_to_ids)

# โหลด Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# สร้าง Class สำหรับ Dataset ของ PyTorch
class NERDataset(Dataset):
    def __init__(self, texts, aspects, tokenizer, labels_to_ids, max_len=128):
        self.texts = texts
        self.aspects = aspects
        self.tokenizer = tokenizer
        self.labels_to_ids = labels_to_ids
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item_idx):
        text = str(self.texts[item_idx])
        aspect = str(self.aspects[item_idx])
        
        # Tokenize ข้อความ
        inputs = self.tokenizer(text,
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True,
                                return_tensors="pt")
        
        input_ids = inputs['input_ids'].squeeze(0)
        attention_mask = inputs['attention_mask'].squeeze(0)
        
        # สร้าง Labels (BIO)
        labels = torch.zeros(self.max_len, dtype=torch.long)
        
        # ค้นหาตำแหน่งของ aspect ใน text
        start_pos = text.find(aspect)
        if start_pos != -1:
            end_pos = start_pos + len(aspect)
            
            # แปลงตำแหน่ง char -> ตำแหน่ง token
            char_to_token_map = []
            for i, _id in enumerate(input_ids):
                if _id not in [tokenizer.cls_token_id, tokenizer.pad_token_id, tokenizer.sep_token_id]:
                    token_span = tokenizer.decode([_id])
                    char_to_token_map.extend([i] * len(token_span))

            if start_pos < len(char_to_token_map) and end_pos <= len(char_to_token_map):
                start_token_idx = char_to_token_map[start_pos]
                end_token_idx = char_to_token_map[end_pos - 1]
                
                # ติดป้าย B-ASPECT และ I-ASPECT
                if start_token_idx <= self.max_len and end_token_idx <= self.max_len:
                    labels[start_token_idx] = self.labels_to_ids[f'B-{aspect}']
                    for i in range(start_token_idx + 1, end_token_idx + 1):
                        labels[i] = self.labels_to_ids[f'I-{aspect}']

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

# สร้าง Dataset
ner_dataset = NERDataset(
    texts=df['review_text'].tolist(),
    aspects=df['aspect'].tolist(),
    tokenizer=tokenizer,
    labels_to_ids=labels_to_ids
)

ner_loader = DataLoader(ner_dataset, batch_size=16, shuffle=True)

# --- สร้างและฝึกโมเดล NER ---
print("\n--- เริ่มต้นการฝึกโมเดล NER (Aspect Extractor) ---")
ner_model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME, 
    num_labels=NUM_NER_LABELS,
    id2label=ids_to_labels,
    label2id=labels_to_ids
).to(DEVICE)

optimizer = AdamW(ner_model.parameters(), lr=5e-5)

ner_model.train()
for epoch in range(2): # ในการแข่งจริง อาจจะใช้ 3-5 epochs
    print(f"NER Epoch {epoch + 1}")
    for batch in ner_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        outputs = ner_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
    print(f"  Loss: {loss.item()}")

print("--- การฝึกโมเดล NER เสร็จสิ้น ---")
# บันทึกโมเดล (แนะนำให้ทำในการแข่ง)
# ner_model.save_pretrained("./ner_model")
# tokenizer.save_pretrained("./ner_model")


Using device: cuda


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\potij\.cache\huggingface\hub\models--airesearch--wangchanberta-base-att-spm-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]


--- เริ่มต้นการฝึกโมเดล NER (Aspect Extractor) ---


model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NER Epoch 1
  Loss: 0.07204373180866241
NER Epoch 2
  Loss: 0.07599621266126633
--- การฝึกโมเดล NER เสร็จสิ้น ---


### Sentiment Classifier

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# --- การตั้งค่าพื้นฐาน ---
# ใช้ MODEL_NAME และ DEVICE เดิม

# โหลดข้อมูลที่สร้างไว้
df = pd.read_csv("train_data.csv")

# --- เตรียมข้อมูลสำหรับ Sentiment Classification ---
# แปลง label (positive, negative, neutral) เป็นตัวเลข
label_encoder = LabelEncoder()
df['sentiment_encoded'] = label_encoder.fit_transform(df['sentiment'])
sentiment_labels_to_ids = {label: i for i, label in enumerate(label_encoder.classes_)}
sentiment_ids_to_labels = {i: label for i, label in enumerate(label_encoder.classes_)}
NUM_SENTIMENT_LABELS = len(sentiment_labels_to_ids)


# สร้าง Class สำหรับ Dataset ของ PyTorch
class SentimentDataset(Dataset):
    def __init__(self, texts, aspects, labels, tokenizer, max_len=256):
        self.texts = texts
        self.aspects = aspects
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item_idx):
        text = str(self.texts[item_idx])
        aspect = str(self.aspects[item_idx])
        label = self.labels[item_idx]
        
        # Format input: [CLS] review [SEP] aspect [SEP]
        # เพื่อบอกให้โมเดลรู้ว่าเรากำลังสนใจ aspect ไหน
        encoded_input = self.tokenizer(
            text, aspect,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoded_input['input_ids'].flatten(),
            'attention_mask': encoded_input['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# สร้าง Dataset
sentiment_dataset = SentimentDataset(
    texts=df['review_text'].tolist(),
    aspects=df['aspect'].tolist(),
    labels=df['sentiment_encoded'].tolist(),
    tokenizer=tokenizer
)

sentiment_loader = DataLoader(sentiment_dataset, batch_size=16, shuffle=True)

# --- สร้างและฝึกโมเดล Sentiment ---
print("\n--- เริ่มต้นการฝึกโมเดล Sentiment Classifier ---")
sentiment_model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_SENTIMENT_LABELS,
    id2label=sentiment_ids_to_labels,
    label2id=sentiment_labels_to_ids
).to(DEVICE)

optimizer = AdamW(sentiment_model.parameters(), lr=5e-5)

sentiment_model.train()
for epoch in range(2): # ในการแข่งจริง อาจจะใช้ 3-5 epochs
    print(f"Sentiment Epoch {epoch + 1}")
    for batch in sentiment_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        outputs = sentiment_model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
    print(f"  Loss: {loss.item()}")

print("--- การฝึกโมเดล Sentiment เสร็จสิ้น ---")
# บันทึกโมเดล
# sentiment_model.save_pretrained("./sentiment_model")



--- เริ่มต้นการฝึกโมเดล Sentiment Classifier ---


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\potij\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Sentiment Epoch 1
  Loss: 0.028676072135567665
Sentiment Epoch 2
  Loss: 0.02075238525867462
--- การฝึกโมเดล Sentiment เสร็จสิ้น ---


### Pipeline และไฟล์ Submission

In [16]:
import pandas as pd
import torch
from tqdm import tqdm

# ทำให้โมเดลอยู่ในโหมดประเมินผล
ner_model.eval()
sentiment_model.eval()

# โหลดข้อมูล test
test_df = pd.read_csv("test_data.csv")

results = []

print("\n--- เริ่มการทำนายผลบน Test Set ---")
# วนลูปในข้อมูล Test ทีละแถว
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    review_id = row['review_id']
    text = row['review_text']

    # --- 1. ใช้ NER Model สกัด Aspect ---
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(DEVICE)
    with torch.no_grad():
        logits = ner_model(**inputs).logits
    
    predictions = torch.argmax(logits, dim=2)
    predicted_labels = [ids_to_labels[p.item()] for p in predictions[0]]
    print(predicted_labels)
    # รวม Token ที่เป็น Aspect เดียวกันเข้าด้วยกัน
    extracted_aspects = set()
    current_aspect = ""
    for token, label in zip(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]), predicted_labels):
        if label.startswith("B-"):
            if current_aspect: # ถ้ามี aspect ก่อนหน้า ให้เก็บก่อน
                extracted_aspects.add(current_aspect.replace(" ", "").replace(" ", " "))
            current_aspect = ids_to_labels[int(label.split('-')[1])] if label.split('-')[1].isdigit() else label.split('-')[1]

        elif label.startswith("I-") and current_aspect:
                pass # ในโค้ดนี้เราจะใช้ชื่อจาก B-tag เลยเพื่อความง่าย
        else:
            if current_aspect:
                extracted_aspects.add(current_aspect.replace(" ", "").replace(" ", " "))
                current_aspect = ""
    if current_aspect:
            extracted_aspects.add(current_aspect.replace(" ", "").replace(" ", " "))
    print(extracted_aspects)
    # --- 2. สำหรับแต่ละ Aspect ที่เจอ, ใช้ Sentiment Model ทำนายความรู้สึก ---
    if not extracted_aspects:
        # ถ้าหา Aspect ไม่เจอเลย อาจจะข้ามไปหรือใส่ค่า default
        pass
    else:
        for aspect in extracted_aspects:
            # เตรียม Input สำหรับ Sentiment Model
            sentiment_inputs = tokenizer(text, aspect, return_tensors="pt", truncation=True, max_length=256).to(DEVICE)
            
            with torch.no_grad():
                sentiment_logits = sentiment_model(**sentiment_inputs).logits
            
            sentiment_prediction_id = torch.argmax(sentiment_logits, dim=1).item()
            predicted_sentiment = sentiment_ids_to_labels[sentiment_prediction_id]
            print(predicted_sentiment)
            # เก็บผลลัพธ์
            results.append({
                'id': review_id,
                'aspect': aspect,
                'sentiment': predicted_sentiment
            })

# สร้าง DataFrame สำหรับส่งประกวด
submission_df = pd.DataFrame(results)

# ตรวจสอบรูปแบบไฟล์
print("\nตัวอย่างผลลัพธ์:")
print(submission_df.head())

# บันทึกเป็น submission.csv
submission_df.to_csv("submission.csv", index=False)

print("\nสร้างไฟล์ submission.csv สำเร็จ!")



--- เริ่มการทำนายผลบน Test Set ---


  1%|          | 6/500 [00:00<00:23, 21.40it/s]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

  3%|▎         | 15/500 [00:00<00:15, 32.33it/s]

set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

...

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

 99%|█████████▉| 497/500 [00:11<00:00, 45.66it/s]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

100%|██████████| 500/500 [00:11<00:00, 43.63it/s]

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
set()
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

In [7]:
submission_df

""


In [8]:
test_df

,review_text,review_id
0,เป็นร้านอาหารแนะนำในวงใน เลยแวะมาทานสักหน่อย\n...,0
1,เป็นร้านที่ตั้งอยุตรงถนนนิมมานเลย เห็นแต่ไกล ร...,1
2,ร้านกาแฟเล็กๆ ในย่านสัมมากร ใช้หลายเมล็ดพันธุ์...,2
3,ร้านนี้เป็นหนึ่งในร้านดังจังหวัดราชบุรี วัตถุด...,3
4,หากคัยผ่านไปมาแถวม. เกษตร ประตู 3 จะเห็นร้านไก...,4
...,...,...
495,ขอขอบคุณเจ้าภาพ(...ขออภัย_สงวนนาม.....)\nพามาท...,495
496,ไอติมโฮมเมทที่มีหลายรสให้เลือก ไปแต่ล่ะวันรสที...,496
497,เป็นบ้านที่รับทำเค้กตามเทศกาลต่างๆ. ไม่มีที่นั...,497
498,ใครผ่านไปผ่านมาที่สายใต้ใหม่ คงเย็นกายสบายใจกั...,498
